In [1]:
import requests
import re
import time
import string
import random
import math
import pandas as pd
from urllib.parse import urlparse
from collections import Counter
from bs4 import BeautifulSoup
from pymystem3 import Mystem

# Задание №1

In [2]:
def crawl(urls):
    links_tree = set()
    for url in urls:
        if len(texts) >= 100:
            return
    
        if url in visited:
            continue
        visited.add(url)
            
        try:
            html = requests.get(url).content.decode()
        except:
            time.sleep(1)
            continue
            
        soup = BeautifulSoup(html)
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text(' ').strip().replace('\xa0', ' ').replace('\n', '').replace('\r', '').replace('\t', '')
        text = re.sub(' +', ' ', text)
    
        if len(text.split(' ')) >= 1000:
            texts[url] = text
            
        parsed = urlparse(url)
        base = f"{parsed.scheme}://{parsed.netloc}"    
        links = re.findall('''<a\s+(?:[^>]*?\s+)?href="([^"]*)"''', html) 
        
        for i, link in enumerate(links):    
            if not urlparse(link).netloc:    
                link_with_base = base + link    
                links[i] = link_with_base
                
        links_tree.update(set(filter(lambda x: 'mailto' not in x, links)))
    
    crawl(links_tree)

In [3]:
texts = {}
visited = set()
start = 'https://m.habr.com/ru/all/'

crawl([start])

In [4]:
index = {i:k for i, k in enumerate(texts)}

In [5]:
with open('index.txt', 'w') as f:
    for i in index:
        f.write(f"{i}: {index[i]}\n")

In [6]:
!cat index.txt

0: https://m.habr.com/ru/post/494504/
1: https://m.habr.com/ru/company/wd/blog/493378/
2: https://m.habr.com/ru/post/494460/
3: https://m.habr.com/ru/post/494474/
4: https://www.youtube.com/channel/UCd_sTwKqVrweTt4oAKY5y4w
5: https://habr.com/ru/all/?mobile=no
6: https://m.habr.com/ru/company/otus/blog/494480/
7: https://m.habr.com/ru/company/wd/blog/493378/comments/
8: https://m.habr.com/ru/company/pt/blog/494472/
9: https://twitter.com/habr_com
10: https://m.habr.com/ru/post/494520/
11: https://m.habr.com/ru/post/494488/
12: https://m.habr.com/ru/company/otus/blog/494476/
13: https://m.habr.com/ru/post/494514/
14: https://m.habr.com/ru/company/pt/blog/494486/
15: https://zen.yandex.ru/habr
16: https://u.tmtm.ru/free
17: https://m.habr.com/ru/post/494478/
18: https://tmtm.ru/megapost
19: https://habr.com#broadcast_posts_week
20: https://t.co/Uq15dFn2Nt
21: https://attack.mitre.org/techniques/T1045/
22: https://habr.com/ru/post/494492/
23: https://www.pcgamingwiki.com/wiki/The_Big_List

# Задание №2

In [7]:
lemmatizer = Mystem()

In [8]:
texts_lemmatized = {}

for url in texts:
    text = texts[url]
    text = text.translate(str.maketrans('', '', string.punctuation)).replace('—', '')
    text = re.sub(' +', ' ', text)
    lemmas = lemmatizer.lemmatize(text)
    texts_lemmatized[url] = ''.join(lemmas).strip()

# Задание №3

In [9]:
docs = [index[k] for k in index]

In [10]:
reverse_index = {}

for url in texts_lemmatized:
    for word in texts_lemmatized[url].split(' '):
        if not word in reverse_index:
            reverse_index[word] = set([url])
        else:
            reverse_index[word].update([url])

In [11]:
logic = {
    '&': lambda x,y: x & y,
    '|': lambda x,y: x | y,
    '!': lambda x: set([doc for doc in docs if doc not in x])
}

def find(query):
    query = query.replace('(', ' ( ').replace(')', ' ) ').replace('!','! ').split()
    
    indexes_words = [i for i,x in enumerate(query) if not x in ['!', '&', '|']]
    for i in indexes_words:
        query[i] = reverse_index[query[i]]
    
    indexes_not = [i for i,x in enumerate(query) if x == '!']
    for i in indexes_not:
        x = query[i+1]
        query[i+1] = logic['!'](x)
        query.pop(i)
    
    indexes_and = [i for i,x in enumerate(query) if x == '&']
    for i in indexes_and:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['&'](x,y)
        query.pop(i)
        query.pop(i)
    
    indexes_or = [i for i,x in enumerate(query) if x == '|']
    for i in indexes_or:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['|'](x,y)
        query.pop(i)
        query.pop(i)
    
    return query[0]

In [12]:
x1 = random.choice(list(reverse_index.keys()))
x2 = random.choice(list(reverse_index.keys()))
x3 = random.choice(list(reverse_index.keys()))

In [13]:
query = f'{x1} | {x2} & !{x3}'
print(query)

наблюдатель | распыляться & !«industrial


In [14]:
find(query)

{'https://habr.com/ru/post/492946/',
 'https://habr.com/ru/post/494284/',
 'https://ru.wikipedia.org/wiki/%D0%92%D1%81%D0%B5%D0%BC%D0%B8%D1%80%D0%BD%D0%B0%D1%8F_%D0%BE%D1%80%D0%B3%D0%B0%D0%BD%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%B7%D0%B4%D1%80%D0%B0%D0%B2%D0%BE%D0%BE%D1%85%D1%80%D0%B0%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F'}

# Задание №4

In [15]:
tfs = {}

for word in reverse_index:
    for doc in docs:
        tokens = texts[doc].split()
        tf = tokens.count(word) / len(tokens)
        if not word in tfs:
            tfs[word] = {}
        tfs[word][doc] = tf

In [16]:
pd.DataFrame(tfs).T

,https://m.habr.com/ru/post/494504/,https://m.habr.com/ru/company/wd/blog/493378/,https://m.habr.com/ru/post/494460/,https://m.habr.com/ru/post/494474/,https://www.youtube.com/channel/UCd_sTwKqVrweTt4oAKY5y4w,https://habr.com/ru/all/?mobile=no,https://m.habr.com/ru/company/otus/blog/494480/,https://m.habr.com/ru/company/wd/blog/493378/comments/,https://m.habr.com/ru/company/pt/blog/494472/,https://twitter.com/habr_com,...,http://admixer.by/,https://habr.com/ru/hub/soft/,https://netacad.com,https://habr.com/ru/post/494492/#comments,https://habr.com/ru/hub/infosecurity/,https://habr.com/ru/company/wd/blog/493378/?mobile=no,https://github.com/reduxjs/redux,https://habr.com/ru/search/?q=%5B%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%5D&target_type=posts,https://habr.com/ru/post/494474/,https://habr.com/ru/post/494478/#habracut
имитированый,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
Cisco,0.009498,0.0,0.0,0.0,0.0,0.000649,0.0,0.0,0.0,0.0,...,0.0,0.0,0.011459,0.0,0.000997,0.0,0.0,0.0,0.0,0.000000
идентичный,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
натуральный,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
хабр,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
QGroundControl,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000505
PixHawk,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
VOLO,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000505
Wunder,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000505


In [17]:
idfs = {}

for word in reverse_index:
    idfs[word] = math.log(len(docs) / len(reverse_index[word]))

In [18]:
pd.DataFrame(idfs,index=['']).T

,
имитированый,3.218876
Cisco,2.120264
идентичный,1.897120
натуральный,2.659260
хабр,0.235722
...,...
QGroundControl,4.605170
PixHawk,4.605170
VOLO,4.605170
Wunder,4.605170


In [19]:
tfidfs = {}

for word in tfs:
    for doc in tfs[word]:
        tfidf = tfs[word][doc] * idfs[word]
        if not word in tfidfs:
            tfidfs[word] = {}
        tfidfs[word][doc] = tfidf

In [20]:
pd.DataFrame(tfidfs).T

,https://m.habr.com/ru/post/494504/,https://m.habr.com/ru/company/wd/blog/493378/,https://m.habr.com/ru/post/494460/,https://m.habr.com/ru/post/494474/,https://www.youtube.com/channel/UCd_sTwKqVrweTt4oAKY5y4w,https://habr.com/ru/all/?mobile=no,https://m.habr.com/ru/company/otus/blog/494480/,https://m.habr.com/ru/company/wd/blog/493378/comments/,https://m.habr.com/ru/company/pt/blog/494472/,https://twitter.com/habr_com,...,http://admixer.by/,https://habr.com/ru/hub/soft/,https://netacad.com,https://habr.com/ru/post/494492/#comments,https://habr.com/ru/hub/infosecurity/,https://habr.com/ru/company/wd/blog/493378/?mobile=no,https://github.com/reduxjs/redux,https://habr.com/ru/search/?q=%5B%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%5D&target_type=posts,https://habr.com/ru/post/494474/,https://habr.com/ru/post/494478/#habracut
имитированый,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
Cisco,0.020139,0.0,0.0,0.0,0.0,0.001377,0.0,0.0,0.0,0.0,...,0.0,0.0,0.024296,0.0,0.002113,0.0,0.0,0.0,0.0,0.000000
идентичный,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
натуральный,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
хабр,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
QGroundControl,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.002323
PixHawk,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
VOLO,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.002323
Wunder,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.002323


# Задание №5

In [21]:
docs_length = {}
for row in pd.DataFrame(tfidfs).iterrows():
    sum_len = 0
    for word in row[1]:
        sum_len += word**2
    docs_length[row[0]] = math.sqrt(sum_len)

In [22]:
query = "правовая защита контента"
query = ''.join(lemmatizer.lemmatize(query)).strip()

tfs_query = {}
for word in query.split():
    for doc in docs:
        tokens = texts[doc].split()
        tf = tokens.count(word) / len(tokens)
        if not word in tfs_query:
            tfs_query[word] = {}
        tfs_query[word][doc] = tf

In [23]:
idfs_query = {}

for word in query.split():
    idfs_query[word] = math.log(len(docs) / len(reverse_index[word]))

In [24]:
tfidfs_query = {}

for word in tfs_query:
    for doc in tfs_query[word]:
        tfidf = tfs_query[word][doc] * idfs_query[word]
        if not word in tfidfs_query:
            tfidfs_query[word] = {}
        tfidfs_query[word][doc] = tfidf

In [25]:
tfidfs_query_flat = pd.DataFrame(tfidfs_query).T.sum().to_dict()

In [26]:
sum_len = 0
for doc in tfidfs_query_flat:
    sum_len += tfidfs_query_flat[doc]**2
query_length = math.sqrt(sum_len)

In [27]:
cos_sim = {}
for doc in docs:
    for word in query.split(' '):
        cs = ((pd.Series(tfidfs_query_flat) * tfs[word][doc]).sum()) / (docs_length[doc] * query_length)
    cos_sim[doc] = cs

In [28]:
Counter(cos_sim).most_common(5)

[('https://m.habr.com/ru/post/494488/', 0.3550683322376472),
 ('https://tmtm.ru/megapost', 0.036782017144031266),
 ('https://m.habr.com/ru/company/wd/blog/493378/comments/',
  0.016721698695944514),
 ('https://habr.com/ru/hub/gadgets/', 0.01652857595533733),
 ('https://m.habr.com/ru/company/pt/blog/494486/', 0.012840283695295754)]